# 1. Load the Discriminator

In [1]:
# Used to create a generator/discriminiator to verify passwords are being one_hot_encoded correctly
G_OPTIMIZE_LEARNING_RATE = 0.0001
D_OPTIMIZE_LEARNING_RATE = 0.00001

# Model monitor for storoing genreated passwords, not needed, can be taken out in future
FOLDER_PATH = "training_sessions"
TRAINING_SESSION = None
TRAINING_SESSION_PATH = None

In [2]:
import os

# Get the current working directory
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

# Check if the last directory in the current path is 'notebooks'
if 'notebooks' in current_dir.split(os.sep):
    # Change the current working directory two levels up
    os.chdir('../../')
    print(f"Changed current working directory to: {os.getcwd()}\n")
else:
    print("Current directory is not inside 'notebooks', no change needed.")


Current working directory: /Users/bambrick/DevCenter/Juypter/PasswordGAN/notebooks/demo
Changed current working directory to: /Users/bambrick/DevCenter/Juypter/PasswordGAN



In [3]:
def one_hot_encode(password):
    # Define the one-hot encoding for each digit
    encoding = []
    for char in password:
        one_hot = [0]*10
        one_hot[int(char)] = 1
        encoding.extend(one_hot)
    return encoding

In [4]:
import tensorflow as tf

class GeneratorLSTMv2(tf.keras.Model):
    def __init__(self, noise_dim=100, **kwargs):
        super(GeneratorLSTMv2, self).__init__(**kwargs)
        
        # Initial dense layer
        self.noise_dim = noise_dim
        self.dense_1 = tf.keras.layers.Dense(256, activation='relu', input_dim=noise_dim,)
        self.batch_norm_1 = tf.keras.layers.BatchNormalization()
        self.dropout_1 = tf.keras.layers.Dropout(0.5)
        
        # LSTM layers for sequence data
        self.reshape_1 = tf.keras.layers.Reshape((1, 256))  # Reshape input for LSTM
        self.lstm_1 = tf.keras.layers.LSTM(128, return_sequences=True)
        self.lstm_2 = tf.keras.layers.LSTM(128)
        self.batch_norm_2 = tf.keras.layers.BatchNormalization()
        self.dropout_2 = tf.keras.layers.Dropout(0.5)
        
        # Output layers
        self.dense_out = tf.keras.layers.Dense(12 * 10, activation='softmax')  # Adjusted to 12 characters, 10 classes each (digits 0-9)
        self.reshape_out = tf.keras.layers.Reshape((12, 10))  # Reshape for output
    
    def call(self, inputs, training=False):
        x = self.dense_1(inputs)
        x = self.batch_norm_1(x, training=training)
        x = self.dropout_1(x, training=training)
        
        x = self.reshape_1(x)
        x = self.lstm_1(x)
        x = self.lstm_2(x)
        x = self.batch_norm_2(x, training=training)
        x = self.dropout_2(x, training=training)
        
        x = self.dense_out(x)
        x = self.reshape_out(x)
        return x

    def get_config(self):
        config = super().get_config()
        config.update({
            'noise_dim': self.noise_dim
        })
        return config
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [5]:
import tensorflow as tf

class DiscriminatorLSTMv2(tf.keras.Model):

    def __init__(self, input_shape=(12, 10), **kwargs):
        super(DiscriminatorLSTMv2, self).__init__(**kwargs)
        self._input_shape = input_shape  # Save input_shape as an attribute    
        self.flatten = tf.keras.layers.Flatten(input_shape=input_shape)
        
        # Layer 1
        self.dense_1 = tf.keras.layers.Dense(256)
        self.leaky_relu_1 = tf.keras.layers.LeakyReLU(alpha=0.2)
        self.dropout_1 = tf.keras.layers.Dropout(0.5)
        self.batch_norm_1 = tf.keras.layers.BatchNormalization()
        
        # Layer 2
        self.dense_2 = tf.keras.layers.Dense(128)
        self.leaky_relu_2 = tf.keras.layers.LeakyReLU(alpha=0.2)
        self.dropout_2 = tf.keras.layers.Dropout(0.5)
        self.batch_norm_2 = tf.keras.layers.BatchNormalization()
        
        # Output layer
        self.dense_out = tf.keras.layers.Dense(1, activation='sigmoid')
        
    def call(self, inputs, training=False):
        x = self.flatten(inputs)
        
        x = self.dense_1(x)
        x = self.leaky_relu_1(x)
        x = self.dropout_1(x, training=training)
        x = self.batch_norm_1(x, training=training)
        
        x = self.dense_2(x)
        x = self.leaky_relu_2(x)
        x = self.dropout_2(x, training=training)
        x = self.batch_norm_2(x, training=training)
        
        x = self.dense_out(x)
        return x
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'input_shape': self._input_shape  # Directly use the input_shape passed during initialization
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [6]:
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.losses import BinaryCrossentropy

def configure_losses_optimizers():
    gen_opt = Adam(learning_rate=G_OPTIMIZE_LEARNING_RATE)
    gen_loss = BinaryCrossentropy()
    
    dis_opt = Adam(learning_rate=D_OPTIMIZE_LEARNING_RATE)
    dis_loss = BinaryCrossentropy()

    return gen_opt, gen_loss, dis_opt, dis_loss

In [7]:
class RanPassGAN(tf.keras.Model): #(Model):
    def __init__(self, generator, discriminator, *args, **kwargs):
        # Pass through args and kwargs to base class 
        super().__init__(*args, **kwargs)
        
        # Create attributes for gen and disc
        self.generator = generator 
        self.discriminator = discriminator 

    def get_config(self):
        return {
            "generator_config": self.generator.get_config(),
            "discriminator_config": self.discriminator.get_config()
        }

    @classmethod
    def from_config(cls, config):
        generator = GeneratorLSTMv2.from_config(config['generator_config'])
        discriminator = DiscriminatorLSTMv2.from_config(config['discriminator_config'])
        
        return cls(generator=generator, discriminator=discriminator)

    def call(self, inputs, training=False):
        generated_passwords = self.generator(inputs, training=training)
        return generated_passwords
        
    def compile(self, g_opt, d_opt, g_loss, d_loss, *args, **kwargs): 
        # Compile with base class
        super().compile(*args, **kwargs)
        
        # Create attributes for losses and optimizers
        self.g_opt = g_opt
        self.d_opt = d_opt
        self.g_loss = g_loss
        self.d_loss = d_loss 
        
    def train_step(self, batch):
        # Get the data 
        real_passwords = batch
        batch_size = tf.shape(real_passwords)[0]  # Dynamically get the batch size

        # Generate noise for the generator
        noise = tf.random.normal([batch_size, 100])

        # Generate fake passwords using the generator
        fake_passwords = self.generator(noise, training=True)
        
        # Train the discriminator
        with tf.GradientTape() as d_tape: 
            # Pass the real and fake passwords to the discriminator model
            yhat_real = self.discriminator(real_passwords, training=True) 
            yhat_fake = self.discriminator(fake_passwords, training=True)
            yhat_realfake = tf.concat([yhat_real, yhat_fake], axis=0)

            # Create labels for real and fakes passwords
            y_realfake = tf.concat([tf.zeros_like(yhat_real), tf.ones_like(yhat_fake)], axis=0)
            
            # Calculate loss - BINARYCROSS 
            total_d_loss = self.d_loss(y_realfake, yhat_realfake)
            
        # Apply backpropagation
        dgrad = d_tape.gradient(total_d_loss, self.discriminator.trainable_variables) 
        self.d_opt.apply_gradients(zip(dgrad, self.discriminator.trainable_variables))
        
        # Train the generator 
        with tf.GradientTape() as g_tape: 
            # Generate some new passwords
            gen_passwords = self.generator(tf.random.normal((128, 100)), training=True)
                                        
            # Create the predicted labels
            predicted_labels = self.discriminator(gen_passwords, training=False)
                                        
            # Calculate loss - trick to training to fake out the discriminator
            total_g_loss = self.g_loss(tf.zeros_like(predicted_labels), predicted_labels) 
            
        # Apply backprop
        ggrad = g_tape.gradient(total_g_loss, self.generator.trainable_variables)
        self.g_opt.apply_gradients(zip(ggrad, self.generator.trainable_variables))
        
        return {"d_loss":total_d_loss, "g_loss":total_g_loss}

In [8]:
import os

import tensorflow as tf
import numpy as np

class ModelMonitor(tf.keras.callbacks.Callback):
    def __init__(self, num_passwords=10, latent_dim=100):
        self.num_passwords = num_passwords
        self.latent_dim = latent_dim
    
    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.uniform((self.num_passwords, self.latent_dim))
        generated_outputs = self.model.generator(random_latent_vectors)

        # Convert the generated softmax outputs into digits
        generated_passwords = [self.softmax_to_digit(output) for output in generated_outputs]

        results_file_path = os.path.join(TRAINING_SESSION_PATH, 'epoch_training_results.md')

        # Save to a file
        with open(results_file_path, 'a') as file:
            file.write(f"\n\n## Epoch {epoch} Results\n")
            for idx, password in enumerate(generated_passwords):
                password_str = ''.join(map(str, password))
                file.write(f"- Generated password {idx}: {password_str}\n")
                print(f"Epoch {epoch}: Generated password {idx}: {password_str}")

    def softmax_to_digit(self, softmax_output):
        return np.argmax(softmax_output, axis=-1)


In [9]:
import numpy as np

def softmax_to_digit(softmax_output):
    return np.argmax(softmax_output, axis=-1)

In [10]:
import os
import tensorflow as tf
from tensorflow.keras.models import load_model


# Number of passwords to generate
num_passwords = 100

# Noise dimension that your generator model expects
latent_dim = 100  # Example value

# Get the loss optimizers
gen_opt, gen_loss, dis_opt, dis_loss = configure_losses_optimizers()

# Load model with custom objects
custom_objects = {
    'RanPassGAN': RanPassGAN,
    'GeneratorLSTMv2': GeneratorLSTMv2,
    'DiscriminatorLSTMv2': DiscriminatorLSTMv2
}

# Load the generator model
model_path = 'resources/models/numeric_only/metal_12digit_pattern4_full_dataset_8_epochs/models/best_model_epoch_0008'
loaded_model = load_model(model_path, custom_objects=custom_objects, compile=False)
loaded_model.compile(gen_opt, dis_opt, gen_loss, dis_loss)

discriminator = loaded_model.discriminator

2023-11-11 22:28:44.700371: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2023-11-11 22:28:44.700399: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2023-11-11 22:28:44.700403: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2023-11-11 22:28:44.700436: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-11 22:28:44.700452: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# 2. Load the password blacklist

In [11]:
def load_text_file(file_path):
    """
    This function loads a text file into a list, where each line is an element in the list.
    
    :param file_path: str, the path to the text file to be loaded
    :return: list of strings, where each string is a line from the file
    """
    with open(file_path, 'r') as file:
        lines = [line.strip() for line in file.readlines()]
    return lines

In [12]:
def password_in_list(password_to_check):

    password_blackist_path = 'resources/data/password_blacklist/10-million-password-list-top-10000.txt'
    password_list = load_text_file(password_blackist_path)

    return password_to_check in password_list

# 3. Generate Secure Password

In [13]:
from password_generator import PasswordGenerator

pw_generator = PasswordGenerator()
pw_generator.secure = True
pw_generator.use_upper = False
pw_generator.use_lower = False
pw_generator.use_digits = True
pw_generator.use_special = False
pw_generator.avoid_ambiguous = False

In [14]:
import tensorflow as tf

#  Check if a password is considered real by the discriminator model.
def check_password_against_model(discriminator, password, certanty_level=0.45):

    # Preprocess the real password using one-hot encoding
    password_encoded = one_hot_encode(password)

    # Convert the encoded passwords to tensors and add a batch dimension
    password_tensor = tf.convert_to_tensor([password_encoded], dtype=tf.float32)

    # Use the discriminator to predict the probability of the password being real
    password_pred = discriminator.predict(password_tensor)

    # Extract the certainty level of the prediction
    password_certainty = password_pred[0][0]

    # Return True if the password certainty is greater than the certainty level threshold
    return password_certainty > certanty_level

In [15]:
import tensorflow as tf

# Generate a secure password using the generator and check if it is considered real by the discriminator
def generate_secure_password(discriminator, certanty_level=0.45):
    while True:
        # Generate a new password
        password = pw_generator.generate()

        # Check if the password is in the blacklist
        if password_in_list(password):
            print("Password is in blacklist, generating a new one...")
            continue  # Skip to the next iteration and generate a new password
            
        # Preprocess the real password using one-hot encoding
        password_encoded = one_hot_encode(password)

        # Convert the encoded passwords to tensors and add a batch dimension
        password_tensor = tf.convert_to_tensor([password_encoded])

        # Use the discriminator to predict the probability of the password being real
        password_pred = discriminator.predict(password_tensor)

        password_certainty = password_pred[0][0]

        # Check if the password is considered real by the discriminator
        if password_certainty > certanty_level:
            # The password is not in the blacklist and is considered real by the discriminator
            return password, password_certainty
        else:
            print("Password didn't pass the discriminator check, generating a new one...")


In [16]:
import tensorflow as tf
from tensorflow.keras.models import load_model

secure_password, password_certainty = generate_secure_password(discriminator)
print(f"Generated password: {secure_password} - {password_certainty}")


1/1 [==============================] - 0s 252ms/step
Generated password: 822127523283 - 0.47853705286979675


2023-11-11 22:28:46.317156: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
